In [57]:
import numpy as np

In [58]:
X = np.load('./data/features.npy')
Y = np.load('./data/labels.npy')

In [59]:
num_samples = X.shape[0]
num_samples
X.shape

(6562, 19, 19, 1)

In [60]:
board_size = 19
input_shape = (board_size,board_size,1)
 
 
# 拿出90%用来训练,10%用于测试
train_num_samples = int(0.9*num_samples)
X_train,X_test = X[:train_num_samples],X[train_num_samples:]
Y_train,Y_test = Y[:train_num_samples],Y[train_num_samples:]

In [82]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D, ZeroPadding2D
from keras.layers.pooling import MaxPool2D

model = Sequential()
# 网络中的第一层是具有32个输出滤波器,内核尺寸3*3的Conv2D层
# 通常，卷积的输出尺寸小于输入。通过添加padding="same"，您可以要求Keras在边缘周围用0来填充矩阵，使得输出具有与输入相同的尺寸
model.add(Conv2D(filters=32,kernel_size=(3,3),activation="relu",padding='same',input_shape=input_shape))
#添加dropout层
model.add(Dropout(rate=0.6))
# 与上一层几乎一样，只是因为不是第一层，不需要传入输入数据形状
model.add(Conv2D(filters=64,kernel_size=(3,3),activation="relu",padding='same'))
# 添加一个最大池化层
model.add(MaxPool2D(pool_size=(2,2)))
# model.add(AveragePooling2D(pool_size=(2,2)))
#添加dropout层
model.add(Dropout(rate=0.6))
# 然后，将上一卷积层的3D输出压平
model.add(Flatten())
# 增加两个Dense层
model.add(Dense(1000,activation="relu"))
#添加dropout层
model.add(Dropout(rate=0.6))
model.add(Dense(board_size*board_size,activation="softmax"))
#model.add(Dense(1))
model.summary()


Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_50 (Conv2D)           (None, 19, 19, 32)        320       
_________________________________________________________________
dropout_7 (Dropout)          (None, 19, 19, 32)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 19, 19, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 9, 64)          0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 9, 9, 64)          0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 5184)              0         
_________________________________________________________________
dense_27 (Dense)             (None, 1000)            

In [83]:
model.compile(optimizer="sgd",loss="categorical_crossentropy",metrics=["accuracy"])
 
#模型训练和评估
model.fit(X_train,Y_train,batch_size=64,epochs=5,verbose=1,validation_data=(X_test,Y_test))
 
score = model.evaluate(X_test,Y_test,verbose=0)
print("loss:",score[0])
print("accuracy:",score[1])

Epoch 1/5


ValueError: in user code:

    /usr/local/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /usr/local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:533 train_step  **
        y, y_pred, sample_weight, regularization_losses=self.losses)
    /usr/local/lib/python3.7/site-packages/tensorflow/python/keras/engine/compile_utils.py:205 __call__
        loss_value = los